In [1]:
import pandas as pd
import numpy as np

In [3]:
!head -n20 ../data/P53_HUMAN_b01.a2m

>P53_HUMAN/1-393
meepqsdpsvepplsqetfsdlwkllpennvlsplpsqamddlmLSPDDIEQWFTEDPGpDEAPRMPEAAPPVAPAPAAP
TPAAPAPAPSWPLSSSVPSQKTYQGSYGFRLGFLHSGTAKSVTCTYSPALNKMFCQLAKTCPVQLWVDSTPPPGTRVRAM
AIYKQSQHMTEVVRRCPHHERCSDSDGLAPPQHLIRVEGNLRVEYLDDRNTFRHSVVVPYEPPEVGSDCTTIHYNYMCNS
SCMGGMNRRPILTIITLEDSSGNLLGRNSFEVRVCACPGRDRRTEEENLRKKGEPHHELPPGSTKRALPNNTSSSPQPKK
KPLDGEYFTLQIRGRERFEMFRELNEALELKDAQAGKEPGGSRAHSSHLKSKKGqstsrhkklmfktegpdsd
>UniRef100_F7DUR2/39-453
....sqssqtseflspevfqhiwdfleqsmdcirmqdadvsdpmWPQYTNLNSMDQQIQnGSSSTSPYNTDHAQNSVTAP
SPYAQPSSTALSPSPAIPSNTDYPGPHSFDVSFQQSSTAKSATWTYSTELKKLYCQIAKTCPIQIKVMTPPPQGAVIRAM
PVYKKAEHVTEVVKRCPNHELSRENEGIAPPSHLIRVEGNSHAQYVEDPITGRQSVLVPYEPPQVGTEFTTVLYNFMCNS
SCVGGMNRRPILIIVTLETRDGQVLGRRCFEARICACPGRDRKADEDSIRKQQVSDSTKNGDGTKRPFRQSTHGIQMKKR
RSPDDELLYLPVRGRETYEMLLKIKESLELMQYLPQHTIETYRQQQQQQHQHLLqkqssmqs...........
>UniRef100_UPI000FFF6C48/41-455
....sqstqtseflspevfqhiwdfleqsmdcirmqdsdlsdpmWPQYTNLNSMDQQIQnGSSSTSPYNTDHAQNSVTAP
SPYAQPSSTALSPSPAIPSNTDYPGPHSFDVSFQQSSTAKSATWTYSTE

## MSA preprocessing

By default, sequences with 50% or more gaps in the alignment and/or positions with less than 70% residue occupancy will be removed. These parameters may be adjusted as needed by the end user.

(1) filter columns that are not gaps in WT sequence

(2) filter sequence with gap cutoff (<50%)

(3) then filter columns also with gap cutoff (<30%) using filtered sequence.

(4) filter sequence so that all sequences have canonical amino acids 'ACDEFGHIKLMNPQRSTVWY'

In [22]:
from Bio import SeqIO

canonical_aas = 'ACDEFGHIKLMNPQRSTVWY' + '-'

a2m = SeqIO.parse('../data/P53_HUMAN_b01.a2m', 'fasta')
ids, msa = [], []
for i, r in enumerate(a2m):
    ids.append(r.id)
    msa.append(list(r.seq.upper().replace('.', '-')))
    
msa = np.array(msa)
ids = np.array(ids)
gap_mask = msa == '-'

print('Initial MSA shape:', msa.shape)

# Filter columns that are not gaps in WT sequence.
is_gap_in_wt_mask = gap_mask[0]
msa = msa[:, ~is_gap_in_wt_mask]
gap_mask = msa == '-'

# Filter sequence with gap cutoff.
is_seq_to_keep_mask = gap_mask.mean(axis=1) < 0.5
msa = msa[is_seq_to_keep_mask]
ids = ids[is_seq_to_keep_mask]
gap_mask = msa == '-'

# Filter columns also with gap cutoff.
is_col_to_keep_mask = gap_mask.mean(axis=0) < 0.3
msa = msa[:, is_col_to_keep_mask]

# Finally, filter sequence so that all sequences have canonical amino acids
is_canonical_aa_mask = [all(aa in canonical_aas for aa in seq) for seq in msa]
msa = msa[is_canonical_aa_mask]
ids = ids[is_canonical_aa_mask]
seqs = [''.join(s) for s in msa]

print('Filtered MSA shape:', msa.shape)
print('Num sequences:', len(seqs))

Initial MSA shape: (3668, 393)
Filtered MSA shape: (3630, 329)
Num sequences: 3630


### Save filtered MSA to a2m file.

In [24]:
from Bio.Seq import Seq

sequences = [SeqIO.SeqRecord(Seq(s), id=id, name=id, description='') for s, id in zip(seqs, ids)]
with open('../data/P53_HUMAN_b01.filtered.a2m', 'w') as outFile:
    SeqIO.write(sequences, handle=outFile, format='fasta')